<a href="https://colab.research.google.com/github/P3aar/Cartorioebac/blob/main/2_PreprocessamentoDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pyspark

In [19]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, max
from pyspark.ml.feature import StringIndexer, MinMaxScaler, PCA, VectorAssembler

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
spark = SparkSession.builder.getOrCreate()

In [24]:
produtos = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/produtos.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pagamentos_pedido.csv', header=True, inferSchema=True)

In [27]:
# valores categoricos
indexador = StringIndexer(inputCol=['categoria_produto'], outputCol='categoria_produto_index')
modelo_indexer = indexador.fit(produtos)
produtos_indexador_df = modelo_indexer.transform(produtos)

produtos_indexador_df.show()

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+-----------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|categoria_produto_index|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+-----------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|                   11.0|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|          

In [30]:
# Normalizacao Maual
min_val, max_val = pagamentos_pedido.select(min('valor_pagamento'), max('valor_pagamento')).first()
pagamentos_normalizado_df = pagamentos_pedido.withColumn('valor_pagamento_normalizado', (col('valor_pagamento') - min_val) / (max_val - min_val))

min_parc, max_parc = pagamentos_normalizado_df.select(min('parcelas_pagamento'), max('parcelas_pagamento')).first()
pagamentos_normalizado_df = pagamentos_normalizado_df.withColumn('parcelas_pagamentos_normalizado', (col('parcelas_pagamento') - min_parc) / (max_parc - min_parc))

pagamentos_normalizado_df.show()

+--------------------+-------------------+--------------+------------------+---------------+---------------------------+-------------------------------+
|           id_pedido|sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|valor_pagamento_normalizado|parcelas_pagamentos_normalizado|
+--------------------+-------------------+--------------+------------------+---------------+---------------------------+-------------------------------+
|b81ef226f3fe1789b...|                  1|   credit_card|                 8|          99.33|       0.007269424652080491|             0.3333333333333333|
|a9810da82917af2d9...|                  1|   credit_card|                 1|          24.39|       0.001784971984941...|           0.041666666666666664|
|25e8ea4e93396b6fa...|                  1|   credit_card|                 1|          65.71|       0.004808958963940492|           0.041666666666666664|
|ba78997921bbcdc13...|                  1|   credit_card|                 8|      

In [33]:
montar_vetor = VectorAssembler(
    inputCols=['peso_produto_g', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm'],
    outputCol='caracteristicas'
)

produtos_caracteristicas_df = montar_vetor.transform(produtos)
produtos_caracteristicas_df.show()

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     caracteristicas|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|[225.0,16.0,10.0,...|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30

In [48]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler


scaler = MinMaxScaler(inputCol='caracteristicas', outputCol='caracteristicas_normalizado')
modelo_scaler = scaler.fit(produtos_caracteristicas_df)

produtos_caracteristicas_normalizado_df = modelo_scaler.transform(produtos_caracteristicas_df)
produtos_caracteristicas_normalizado_df.show(truncate=False)

+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------------------------------------------------------------------+
|id_produto                      |categoria_produto               |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|caracteristicas         |caracteristicas_normalizado                                                        |
+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------------------------------------------------------------------+
|1e9e8ef04dbcff4541ed26657ea

In [39]:
# Montar vetor de características, ignorando linhas com valores inválidos
montar_vetor = VectorAssembler(
    inputCols=['peso_produto_g', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm'],
    outputCol='caracteristicas',
    handleInvalid="skip" # Ignorar linhas com valores nulos
)

produtos_caracteristicas_df = montar_vetor.transform(produtos)
produtos_caracteristicas_df.show()

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     caracteristicas|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|[225.0,16.0,10.0,...|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30

In [36]:
# Normalizar o vetor de características
scaler = MinMaxScaler(inputCol='caracteristicas', outputCol='caracteristicas_normalizado')
modelo_scaler = scaler.fit(produtos_caracteristicas_df)

produtos_caracteristicas_normalizado_df = modelo_scaler.transform(produtos_caracteristicas_df)
produtos_caracteristicas_normalizado_df.show(truncate=False)

+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------------------------------------------------------------------+
|id_produto                      |categoria_produto               |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|caracteristicas         |caracteristicas_normalizado                                                        |
+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------------------------------------------------------------------+
|1e9e8ef04dbcff4541ed26657ea

In [52]:
# Reduzir dimensionalidade
pca = PCA(k=2, inputCol='caracteristicas_normalizado', outputCol='caracteristicas_pca')
modelo_pca = pca.fit(produtos_caracteristicas_normalizado_df)
produtos_caracteristicas_reduzida_df = modelo_pca.transform(produtos_caracteristicas_normalizado_df)

produtos_caracteristicas_reduzida_df.show(truncate=False)

+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------------------------------------------------------------------+--------------------------------------------+
|id_produto                      |categoria_produto               |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|caracteristicas         |caracteristicas_normalizado                                                        |caracteristicas_pca                         |
+--------------------------------+--------------------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+-----------------------

In [54]:
produtos_caracteristicas_reduzida_df.write.mode('overwrite').option('header', 'true').parquet('drive/MyDrive/Planilha excel ANALISTA DE DADOS/produtos_preprocessados_parquet')

df = spark.read.option('header', 'true').parquet('drive/MyDrive/Planilha excel ANALISTA DE DADOS/produtos_preprocessados_parquet')
df.show(5)
df.printSchema()

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+---------------------------+--------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|     caracteristicas|caracteristicas_normalizado| caracteristicas_pca|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+--------------------+---------------------------+--------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|[225.0,16.0,10.0,...|      

In [55]:
pagamentos_normalizado_df.write.mode('overwrite').option('header', 'true').parquet('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pagamentos_pedido_preprocessados_parquet')

spark.read.option('header', 'true').parquet('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pagamentos_pedido_preprocessados_parquet').show(5)

+--------------------+-------------------+--------------+------------------+---------------+---------------------------+-------------------------------+
|           id_pedido|sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|valor_pagamento_normalizado|parcelas_pagamentos_normalizado|
+--------------------+-------------------+--------------+------------------+---------------+---------------------------+-------------------------------+
|b81ef226f3fe1789b...|                  1|   credit_card|                 8|          99.33|       0.007269424652080491|             0.3333333333333333|
|a9810da82917af2d9...|                  1|   credit_card|                 1|          24.39|       0.001784971984941...|           0.041666666666666664|
|25e8ea4e93396b6fa...|                  1|   credit_card|                 1|          65.71|       0.004808958963940492|           0.041666666666666664|
|ba78997921bbcdc13...|                  1|   credit_card|                 8|      

In [56]:
spark.stop()